# Initialization

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline

In [ ]:
ROOT = '../input/'  # ROOT of our data. You can change this on your environment. 
train_raw = pd.read_json(ROOT+'train.json')
test_raw = pd.read_json(ROOT+'test.json')

Let's take a quick glance at our data.

In [ ]:
train_raw.sample(5).T

I have a bad feeling, maybe I have to model a RNN(for description processing) along with a CNN(for photos) eventually.

What about images?

In [ ]:
Image.open(ROOT+'/images_sample/6811957/6811957_33d08c8dc440c89bccc8d9889c5485a6.jpg')

Nice apartment indeed! As we aren't fully prepared for CNN, we will deal with these photos later.

# Explorations

How many buildings and managers?

In [ ]:
print('Total: %s' % train_raw.shape[0])
print(train_raw[['building_id', 'listing_id', 'manager_id']].T.apply(lambda row: row.nunique(), axis=1))

Now let's checkout our label: `interest_level`

In [ ]:
train_raw.groupby('interest_level').listing_id.count().loc[['low', 'medium', 'high']].plot(kind='bar')
plt.xticks(rotation='horizontal')
plt.show()

How about bathrooms and bedrooms?

In [ ]:
fig, axes = plt.subplots(1,2)
train_raw.groupby('bathrooms').listing_id.count().plot(kind='bar', ax=axes[0])
train_raw.groupby('bedrooms').listing_id.count().plot(kind='bar', ax=axes[1])
plt.show()

`Features` contains a big list of features, is there anything unique? Can we build a relational database on these `features`?

In [ ]:
features = pd.DataFrame({'feature': [j for i in train_raw.features.values for j in i]})
features['dummy'] = 1
features.groupby('feature').count().sort_values('dummy', ascending=False)

There are many special, unique features! We will deal with that later.

How about price? Maybe there are some outliers and skewness.

In [ ]:
print(train_raw.price.describe(percentiles=[0.01, 0.99]))

So there are outliers, let's deal with that.

In [ ]:
price_lower_bound = 1475.00
price_higher_bound = 13000.00
train_raw.query('%s<=price and price<=%s' % (price_lower_bound, price_higher_bound)).price.hist(bins=50)
plt.show()

Next, let's remove skewness using BoxCox transform.

In [ ]:
# remove skewness
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler
data = train_raw.query('%s<=price and price<=%s' % (price_lower_bound, price_higher_bound)).price.values.reshape(-1, 1)
price_scaler = MinMaxScaler(feature_range=(1, 2)).fit(data.astype(float))
d, price_bc_coeff = boxcox(price_scaler.transform(data))
price_bc_coeff = price_bc_coeff[0]
print('BoxCox lambda: %s' % price_bc_coeff)
plt.hist(d, bins=50)
plt.show()

Hooray! Price seems to be okay now.